국내 주식 데이터 수집
이번 장에서는 극내 주식 데이터 중 주식티커와 섹터별 구성종목 및 퀀트 투자를 위한 핵심 데이터인 수정주가, 재무제표, 가치지표를 크롤링하는 방법을 알아보겠다.

최근 영업일 기준 데이터 받기
POST 방식으로 금융 데이터를 제공하는 일부 사이트에서는 쿼리 항목에 특정 날짜를 입력하면(예:20221230) 해당일의 데이터를 다운로드 할 수 있으며, 최근 영업일 날짜를 입력하면 가장 최근의 데이터를 받을 수 있다. 따라서 최근 영업일에 해당하는 항목을 매번 수기로 입력하기 보다는 자동으로 반영되게 할 필요가 있다.

네이버 금융의 [국내증시 → 증시자금동향]에는 이전 2영업일에 해당하는 날짜가 있으며, 자동으로 날짜가 업데이트된다. 따라서 해당 부분을 크롤링 한후 날짜에 해당하는 쿼리 항목에 사용하면 된다.

In [1]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 22.3 MB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 34.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


https://finance.naver.com/sise/sise_deposit.nhn

개발자도구 화면을 이용해 해당 데이터가 있는 부분을 확인해보면 [클래스가 subtop_sise_graph2인 div 태그 → 클래스가 subtop_chart_note인 ul 태그 → li 태그 → 클래스가 tah인 span 태그]에 위치해 있다는 걸 알 수 있다. 이를 이용해 해당 데이터를 크롤링한다.

In [7]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content, 'html.parser')
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2024.10.11



페이지 주소를 입력한다.
get() 함수를 통해 해당 페이지 내용을 받아온다.
BeautifulSoup() 함수를 이용해 해당 페이지의 HTML 내용을 BeautifulSoup 객체로 만든다.
select_one() 메서드를 통해 해당 태그의 데이터를 추출하며, text 메서드를 이용해 텍스트 데이터만을 추출한다.
위 과정을 통해 | yyyy.mm.dd 형식의 데이터가 선택된다. 이 중 숫자 부분만을 뽑아 yyyymmdd 형태로 만들어주도록 한다.

In [8]:

import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

20241011



1. findall() 메서드 내에 정규 표현식을 이용해 숫자에 해당하는 부분만을 추출한다. '[0-9]+' 는 모든 숫자를 의미하는 표현식이다.
2. join() 함수를 통해 숫자를 합쳐준다.
이를 통해 우리가 원하는 yyyymmdd 형태의 날짜가 만들어졌다. 해당 데이터를 최근 영업일이 필요한 곳에 사용하면 된다.

한국거래소의 업종분류 현황 및 개별지표 크롤링
주식 관련 데이터를 구하기 위해 가장 먼저 해야하는 일은 어떠한 종목들이 상장되어 있는가에 대한 정보를 구하는 것이다. 한국거래소에서 제공하는 업종분류 현황과 개별종목 지표 데이터를 이용하면 매우 간단하게 해당 정보를 수집할 수 있다.

KRX 정보데이터시스템 http://data.krx.co.kr/ 에서 [기본통계 → 주식 → 세부안내] 부분
[12025] 업종분류 현황: http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506
[12021] 개별종목: http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020502
해당 데이터들을 크롤링이 아닌 [Excel] 버튼을 클릭해 엑셀 파일로 받을 수도 있다. 그러나 매번 엑셀 파일을 다운로드하고 이를 불러오는 작업은 상당히 비효율적이며, 크롤링을 이용한다면 해당 데이터를 파이썬에서 바로 불러올 수 있다.

업종분류 현황 크롤링
먼저 업종분류 현황에 해당하는 페이지에 접속하여 F12를 눌러 개발자도구 화면을 열고 [다운로드] 버튼을 클릭한 후 [CSV]를 누른다. 개발자도구 화면의 [Network] 탭에는 {numref}krx_1와 같이 generate.cmd와 download.cmd 두 가지 항목이 생긴다.

거래소에서 엑셀 혹은 CSV 데이터를 받는 과정은 다음과 같다.

http://data.krx.co.kr/comm/fileDn/download_excel/download.cmd 에 원하는 항목을 쿼리로 발송하면 해당 쿼리에 해당하는 OTP(generate.cmd)를 받는다.
부여받은 OTP를 http://data.krx.co.kr/ 에 제출하면 이에 해당하는 데이터(download.cmd)를 다운로드한다.
먼저 1번 단계를 살펴보자. [Headers] 탭의 'General' 항목 중 'Request URL' 부분이 원하는 항목을 제출할 주소다. [Payload] 탭의 Form Data에는 우리가 원하는 항목들이 적혀 있다. 이를 통해 POST 방식으로 데이터를 요청하면 OTP를 받음을 알 수 있다.

OTP 생성 부분
다음으로 2번 단계를 살펴보자. 'General' 항목의 'Request URL'은 OTP를 제출할 주소다. 'Form Data'의 OTP는 1번 단계를 통해 부여받은 OTP에 해당한다. 이 역시 POST 방식으로 데이터를 요청하며 이를 통해 CSV 파일을 받아온다.

위 과정 중 OTP를 받아오는 과정을 코드로 나타내면 다음과 같다

In [12]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)










 
 


	
	
		<html>
<head>
  <title>Error - KRX | Market Data System</title>
  <meta charset="utf-8">
  <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
  <meta http-equiv="X-UA-Compatible" content="IE=Edge">
  <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
  <meta name="author" content="?????" />
  <meta name="description" content="??·????? ????(Marketdata), ?????, ??????(SMILE) ? ?????? ?????? ???? ?? ???" />
  <meta name="keywords" content="???????? ?????? ????? ?? KRX?? ??????? ???? ?? ???????? KRX ???????? ??? ?????? ???????? ?????????? ?????" />
  <meta property="og:type" content="website" />
  <meta property="og:title" content="KRX Market Data System" />
  <meta property="og:description" content="??·????? ????(Marketdata), ?????, ??????(SMILE) ? ?????? ?????? ???? ?? ???" />
  <meta name="naver-site-verification" content="222ad2f14a7a611c5494c898d311a8d7b61ba13e"/>
  <meta name="robots" content="index,follow" >
 

gen_otp_url 에 원하는 항목을 제출할 URL을 입력한다.
개발자도구 화면에 있는 쿼리 내용들을 딕셔너리 형태로 입력한다. 이 중 mktId의 'STK'는 코스피에 해당하며, 코스닥 데이터를 받고자 할 경우 'KSQ'를 입력하면 된다.
영업일을 뜻하는 trdDd에는 위에서 구한 최근 영업일 데이터를 입력한다.
헤더 부분에 리퍼러(Referer)를 추가한다. 리퍼러란 링크를 통해서 각각의 웹사이트로 방문할 때 남는 흔적이다. 거래소 데이터를 다운로드하는 과정을 살펴보면 첫 번째 URL에서 OTP를 부여받고, 이를 다시 두번째 URL에 제출했다. 그런데 이러한 과정의 흔적이 없이 OTP를 바로 두번째 URL에 제출하면 서버는 이를 로봇으로 인식해 데이터를 주지 않는다. 따라서 헤더 부분에 우리가 거쳐온 과정을 흔적으로 남겨야 데이터를 받을 수 있다. 이러한 리퍼러 주소는 개발자도구 화면에서도 확인할 수 있다({numref}referer).
post() 함수를 통해 해당 URL에 쿼리를 전송하면 이에 해당하는 데이터를 받으며, 이 중 텍스트에 해당하는 내용만 불러온다.

위의 과정을 거쳐 생성된 OTP를 제출하면, 우리가 원하는 데이터를 다운로드할 수 있다.

In [14]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk

,<HTML><HEAD>
0,<TITLE>Access Denied</TITLE>
1,</HEAD><BODY>
2,<H1>Access Denied</H1>
3,"You don't have permission to access ""http&#58;..."
4,Reference&#32;&#35;18&#46;3e0ed317&#46;1728991...
5,<P>https&#58;&#47;&#47;errors&#46;edgesuite&#4...
6,</BODY>
7,</HTML>
